In [4]:
import pandas as pd
filepath = 'candyhierarchy2017.csv'
df = pd.read_csv(filepath)

In [5]:
df.shape
df.head(10)
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: float64(4), int64(1), object(115)
memory usage: 2.3+ MB


Internal ID                    0
Q1: GOING OUT?               110
Q2: GENDER                    41
Q3: AGE                       84
Q4: COUNTRY                   64
                            ... 
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
Length: 120, dtype: int64

the data contains alot of missing values, candy rating columns are stored correctly "JOY", "MEH","DESPAIR"

In [6]:
#df = df.drop(columns=['internal id'])

In [7]:
def simplify_name(name):
    name = name.lower().strip()
    if 'q6 |' in name:
        name = name.split('|')[-1].strip()
    elif ':' in name:
        name = name.split(':')[-1].strip()
    while '__' in name:
        name = name.replace('__', '_')
    return name.strip('_')

df.columns = [simplify_name(col) for col in df.columns]

In [8]:
min_non_null = int(len(df) * 0.20)
df = df.dropna(axis=1, thresh=min_non_null)

In [9]:
df.shape

(2460, 115)

In [10]:
min_non_null = int(len(df.columns) * 0.20)
df = df.dropna(axis=0, thresh=min_non_null)

In [11]:
df.shape

(1796, 115)

In [12]:
df.columns

Index(['internal id', 'going out?', 'gender', 'age', 'country',
       'state, province, county, etc', '100 grand bar',
       'anonymous brown globs that come in black and orange wrappers\t(a.k.a. mary janes)',
       'any full-sized candy bar', 'black jacks',
       ...
       'whatchamacallit bars', 'white bread', 'whole wheat anything',
       'york peppermint patties', 'joy other', 'despair other', 'dress', 'day',
       'media [science]', 'click coordinates (x, y)'],
      dtype='object', length=115)

In [13]:
df['going out?'] = df['going out?'].fillna('No')

In [14]:
df['gender'].value_counts()

gender
Male                  1082
Female                 618
I'd rather not say      60
Other                   26
Name: count, dtype: int64

In [15]:
unspecified_list = ['I\'d rather not say', 'Other']
df['gender'] = df['gender'].replace(unspecified_list, 'Unspecified').fillna('Unspecified')

In [16]:
df['age'] = pd.to_numeric(df['age'], errors='coerce')
age_median = df['age'].median()
df['age'] = df['age'].fillna(age_median).astype(int)

In [17]:
df['country'].value_counts()

country
USA                               534
United States                     388
usa                               160
Canada                            123
US                                101
                                 ... 
United Statea                       1
subscribe to dm4uz3 on youtube      1
USA USA USA!!!!                     1
I don't know anymore                1
Fear and Loathing                   1
Name: count, Length: 100, dtype: int64

In [18]:
def simplify_country(country):
    if pd.isna(country):
        return None
    country = str(country).lower().strip()
    if 'united kingdom' in country or 'uk' in country or 'scotland' in country or 'england' in country:
        return 'United Kingdom'
    if 'united states' in country or 'usa' in country or 'us' in country or 'america' in country or 'u.s.' in country or'u.s.a.' in country:
        return 'United States'
    if 'canada' in country:
        return 'Canada'
    return country.title()


In [19]:
df['country'] = df['country'].apply(simplify_country).fillna('Unknown')

In [20]:
df['country'].value_counts()

country
United States                     1524
Canada                             158
United Kingdom                      22
Unknown                             21
Germany                              7
Netherlands                          4
France                               3
Ireland                              3
Japan                                3
United State                         2
Switzerland                          2
Denmark                              2
Mexico                               2
Unites States                        2
Murica                               1
U S                                  1
Costa Rica                           1
Insanity Lately                      1
United Staes                         1
Uae                                  1
North Carolina                       1
Earth                                1
Europe                               1
A                                    1
Pittsburgh                           1
'Merica          

In [21]:
keyword = 'state'
found_columns = [col for col in df.columns if keyword.lower() in col.lower()]
print(found_columns)

['state, province, county, etc']


In [22]:
df['state, province, county, etc'].value_counts()

state, province, county, etc
California                                        108
CA                                                 64
Texas                                              44
Illinois                                           42
Oregon                                             39
                                                 ... 
Connecticut                                         1
ill                                                 1
Psychotic                                           1
New York County, New York City, New York State      1
West Yorkshire                                      1
Name: count, Length: 431, dtype: int64

In [23]:
df['state, province, county, etc'] = df['state, province, county, etc'].str.lower().str.strip().fillna('Unknown')

In [24]:
df['state, province, county, etc'].value_counts()

state, province, county, etc
california                                            137
ca                                                     85
ontario                                                57
illinois                                               55
texas                                                  51
                                                     ... 
chicago, il                                             1
st. augustine florida                                   1
the democratic people's republic of north carolina      1
pennsylvania, northampton couny, easton                 1
psychotic                                               1
Name: count, Length: 276, dtype: int64

In [25]:
df.columns[6:]

Index(['100 grand bar',
       'anonymous brown globs that come in black and orange wrappers\t(a.k.a. mary janes)',
       'any full-sized candy bar', 'black jacks', 'bonkers (the candy)',
       'bonkers (the board game)', 'bottle caps', 'box'o'raisins',
       'broken glow stick', 'butterfinger',
       ...
       'whatchamacallit bars', 'white bread', 'whole wheat anything',
       'york peppermint patties', 'joy other', 'despair other', 'dress', 'day',
       'media [science]', 'click coordinates (x, y)'],
      dtype='object', length=109)

In [26]:
def detect_candy_col(s):
    values = s.dropna().astype(str).str.strip().str.upper()
    check = values.isin(['JOY', 'MEH', 'DESPAIR'])
    return check.sum() >= max(5, int(0.3 * len(values)))

candy_cols = [i for i in df.columns if detect_candy_col(df[i])]

In [27]:
candy_cols

['100 grand bar',
 'anonymous brown globs that come in black and orange wrappers\t(a.k.a. mary janes)',
 'any full-sized candy bar',
 'black jacks',
 'bonkers (the candy)',
 'bonkers (the board game)',
 'bottle caps',
 "box'o'raisins",
 'broken glow stick',
 'butterfinger',
 'cadbury creme eggs',
 'candy corn',
 'candy that is clearly just the stuff given out for free at restaurants',
 'caramellos',
 'cash, or other forms of legal tender',
 'chardonnay',
 'chick-o-sticks (we don’t know what that is)',
 'chiclets',
 'coffee crisp',
 'creepy religious comics/chick tracts',
 'dental paraphenalia',
 'dots',
 'dove bars',
 'fuzzy peaches',
 'generic brand acetaminophen',
 'glow sticks',
 'goo goo clusters',
 "good n' plenty",
 'gum from baseball cards',
 'gummy bears straight up',
 'hard candy',
 'healthy fruit',
 'heath bar',
 "hershey's dark chocolate",
 'hershey’s milk chocolate',
 "hershey's kisses",
 'hugs (actual physical hugs)',
 'jolly rancher (bad flavor)',
 'jolly ranchers (good f

In [28]:
def candy_col_cleaning (x):
    if pd.isna(x):
        return pd.NA
    y = str(x).strip().upper()
    if y in ["JOY", "MEH", "DESPAIR"]:
        return y
    return pd.NA

In [29]:
for i in candy_cols:
    df[i] = df[i].apply(candy_col_cleaning)

In [30]:
df.shape

(1796, 115)

In [31]:
df['dress'].value_counts(dropna=False)

dress
White and gold    1077
Blue and black     634
NaN                 85
Name: count, dtype: int64

In [32]:
df['dress'] = df['dress'].fillna("Unknown")

In [33]:
df['dress'].value_counts(dropna=False)

dress
White and gold    1077
Blue and black     634
Unknown             85
Name: count, dtype: int64

In [34]:
df['day'].value_counts(dropna=False)

day
Friday    1090
Sunday     641
NaN         65
Name: count, dtype: int64

In [35]:
df['day'] = df['day'].fillna("Unknown")

In [36]:
df['day'].value_counts(dropna=False)

day
Friday     1090
Sunday      641
Unknown      65
Name: count, dtype: int64

In [37]:
df.isna().sum().sort_values(ascending=False)

despair other                   1080
joy other                        886
media [science]                  438
joyjoy (mit iodine!)             362
maynards                         360
                                ... 
going out?                         0
state, province, county, etc       0
age                                0
day                                0
dress                              0
Length: 115, dtype: int64

In [38]:
df = df.drop(columns=['despair other'])

In [39]:
df = df.drop(columns=['joy other'])

In [40]:
df.isna().sum().sort_values(ascending=False)

media [science]                 438
joyjoy (mit iodine!)            362
maynards                        360
reggie jackson bar              350
bonkers (the board game)        343
                               ... 
state, province, county, etc      0
age                               0
country                           0
day                               0
dress                             0
Length: 113, dtype: int64

In [41]:
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [42]:
for i in candy_cols:
    print(i, df[i].unique())

100 grand bar ['MEH' 'JOY' <NA> 'DESPAIR']
anonymous brown globs that come in black and orange wrappers	(a.k.a. mary janes) ['DESPAIR' 'MEH' 'JOY' <NA>]
any full-sized candy bar ['JOY' 'MEH' <NA> 'DESPAIR']
black jacks ['MEH' 'DESPAIR' <NA> 'JOY']
bonkers (the candy) ['DESPAIR' 'MEH' <NA> 'JOY']
bonkers (the board game) ['DESPAIR' <NA> 'MEH' 'JOY']
bottle caps ['DESPAIR' 'MEH' 'JOY' <NA>]
box'o'raisins ['DESPAIR' 'MEH' 'JOY' <NA>]
broken glow stick ['DESPAIR' 'JOY' 'MEH' <NA>]
butterfinger ['DESPAIR' 'MEH' 'JOY' <NA>]
cadbury creme eggs ['MEH' 'DESPAIR' 'JOY' <NA>]
candy corn ['MEH' 'DESPAIR' 'JOY' <NA>]
candy that is clearly just the stuff given out for free at restaurants ['DESPAIR' 'MEH' 'JOY' <NA>]
caramellos ['MEH' 'JOY' <NA> 'DESPAIR']
cash, or other forms of legal tender ['JOY' 'MEH' 'DESPAIR' <NA>]
chardonnay ['MEH' 'JOY' <NA> 'DESPAIR']
chick-o-sticks (we don’t know what that is) ['DESPAIR' 'JOY' 'MEH' <NA>]
chiclets ['DESPAIR' 'MEH' <NA> 'JOY']
coffee crisp ['DESPAIR' 'MEH' '

In [43]:
for col in ['gender', 'dress', 'day', 'country']:
    print(col, df[col].unique())


gender ['Male' 'Female' 'Unspecified']
dress ['White and gold' 'Unknown' 'Blue and black']
day ['Sunday' 'Friday' 'Unknown']
country ['United States' 'Unknown' 'Canada' 'Murica' 'United Kingdom'
 'United Staes' 'Uae' 'France' 'Mexico' 'Unites States' 'North Carolina'
 'Netherlands' 'Europe' 'Earth' 'U S' 'Costa Rica' 'Cascadia'
 'Insanity Lately' 'Greece' "'Merica" 'Pittsburgh' 'United State' 'A'
 'Can' 'Canae' 'New York' 'Trumpistan' 'Ireland' 'United Sates' 'Korea'
 'California' 'Japan' 'Iceland' 'Denmark' 'Switzerland' 'Ahem....Amerca'
 'South Korea' 'New Jersey' 'United Stated' 'Germany' 'United Statss'
 'Endland' 'Atlantis' 'Murrika' 'Alaska' 'Singapore' 'Taiwan' 'China'
 'Spain' 'Narnia' 'U S A' 'United Statea' 'Subscribe To Dm4Uz3 On Youtube'
 "I Don'T Know Anymore" 'Fear And Loathing']


In [44]:
df.head(10)

,internal id,going out?,gender,age,country,"state, province, county, etc",100 grand bar,anonymous brown globs that come in black and orange wrappers\t(a.k.a. mary janes),any full-sized candy bar,black jacks,...,"vials of pure high fructose corn syrup, for main-lining into your vein",vicodin,whatchamacallit bars,white bread,whole wheat anything,york peppermint patties,dress,day,media [science],"click coordinates (x, y)"
1,90272821,No,Male,44,United States,nm,MEH,DESPAIR,JOY,MEH,...,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,1.0,"(84, 25)"
3,90272840,No,Male,40,United States,or,MEH,DESPAIR,JOY,MEH,...,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,1.0,"(75, 23)"
4,90272841,No,Male,23,United States,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,1.0,"(70, 10)"
5,90272852,No,Male,42,Unknown,Unknown,JOY,DESPAIR,JOY,<NA>,...,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,Unknown,Unknown,1.0,"(75, 23)"
7,90272854,No,Male,33,Canada,ontario,JOY,DESPAIR,JOY,DESPAIR,...,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Blue and black,Friday,1.0,"(55, 5)"
8,90272858,No,Male,40,Canada,ontario,JOY,DESPAIR,JOY,MEH,...,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,Blue and black,Sunday,1.0,"(76, 24)"
9,90272859,No,Female,53,United States,wa,MEH,DESPAIR,JOY,MEH,...,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,White and gold,Sunday,1.0,"(70, 28)"
11,90272865,No,Male,56,Canada,quebec,JOY,MEH,JOY,MEH,...,DESPAIR,MEH,MEH,JOY,DESPAIR,MEH,White and gold,Friday,1.0,"(73, 24)"
12,90272866,No,Male,64,United States,ny,MEH,MEH,JOY,MEH,...,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,White and gold,Sunday,1.0,"(77, 24)"
13,90272867,Yes,Male,43,Murica,california,JOY,DESPAIR,JOY,MEH,...,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,White and gold,Sunday,1.0,NaN


 -i dropped internal id column because it serves no purpose
 -dropped rows,columns with alot of missing values
 -renamed columns for easy access
 -cleaned gender column and changed 'i'd rather not say','other' to unspecified
 -converted age column to numeric
 -for country column i grouped misspelling and filled all missing values with unknown
 -identified candy columns and filled missing values with nan
 -dropped joyother, despairother columns
 -for dress,day columns filled missing values with unknown

In [51]:
candy = [col for col in df.columns if detect_candy_col(df[col])]
rating = {'JOY': 1,'MEH': 0,'DESPAIR': -1,'No Opinion': 0}
mean_scores = df[candy].replace(rating).mean().sort_values(ascending=False)

In [52]:
most_loved = mean_scores.head(10).to_frame(name='Average_Score')
most_hated = mean_scores.tail(10).to_frame(name='Average_Score').sort_values(by='Average_Score', ascending=True)

In [55]:
print(most_hated)
print(most_loved)

                                                   Average_Score
broken glow stick                                      -0.912721
white bread                                            -0.828538
real housewives of orange county season 9 blue-ray     -0.813708
gum from baseball cards                                -0.813073
kale smoothie                                           -0.77331
dental paraphenalia                                     -0.75905
candy that is clearly just the stuff given out ...     -0.726964
whole wheat anything                                    -0.68418
creepy religious comics/chick tracts                   -0.660592
spotted dick                                           -0.641318
                                     Average_Score
any full-sized candy bar                  0.862668
reese’s peanut butter cups                0.787521
kit kat                                    0.78083
cash, or other forms of legal tender       0.77191
twix                          